In [13]:
import torch
import torch.utils.dlpack
import open3d as o3d
import numpy as np

## Tensor creation

In [14]:
# From list
a = o3d.core.Tensor([0, 1, 2])
print(a)

# Dtype inferred from list
a_float = o3d.core.Tensor([0.0, 1.0, 2.0])
print(a_float)

# From Numpy
a = o3d.core.Tensor(np.array([0, 1, 2]))
print(a)

# Specify dtype
a = o3d.core.Tensor(np.array([0, 1, 2]), dtype=o3d.core.Dtype.Float64)
print(a)

# Specify device
# Specify dtype
a = o3d.core.Tensor(np.array([0, 1, 2]), device=o3d.core.Device("CUDA:0"))
print(a)

[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x7f9ee3e1a3e0]
[0.0 1.0 2.0]
Tensor[shape={3}, stride={1}, Float64, CPU:0, 0x7f9ee8e698b0]
[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x7f9ee8e67470]
[0.0 1.0 2.0]
Tensor[shape={3}, stride={1}, Float64, CPU:0, 0x7f9ee8e69b00]
[0 1 2]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x7f9ee8e69890]


## Properties of a tensor

In [15]:
vals = np.array((range(24))).reshape(2, 3, 4)
a = o3d.core.Tensor(np.array(vals), 
               dtype=o3d.core.Dtype.Float64,
               device=o3d.core.Device("CUDA:0"))
print(f"a.shape: {a.shape}")
print(f"a.strides: {a.strides}")
print(f"a.dtype: {a.dtype}")
print(f"a.device: {a.device}")
print(f"a.ndim: {a.ndim}")

a.shape: {2, 3, 4}
a.strides: {12, 4, 1}
a.dtype: Dtype.Float64
a.device: CPU:0
a.ndim: 3


## Copy & device transfer

In [18]:
# Host -> Device
a_cpu = o3d.core.Tensor([0, 1, 2])
a_gpu = a_cpu.cuda(0)
print(a_gpu)

# Device -> Device
a_gpu = o3d.core.Tensor([0, 1, 2], device=o3d.core.Device("CUDA:0"))
a_cpu = a_gpu.cpu()
print(a_cpu)

# Device -> Host
a_gpu_0 = o3d.core.Tensor([0, 1, 2], device=o3d.core.Device("CUDA:0"))
a_gpu_1 = a_gpu_0.cuda(1)
print(a_gpu_1)

RuntimeError: [1;31m[Open3D ERROR] CUDA is not available, cannot copy Tensor.[0;m

## Type casting

In [19]:
# E.g. float -> int
a = o3d.core.Tensor([0.1, 1.5, 2.7])
b = a.to(o3d.core.Dtype.Int32)
print(a)
print(b)

[0.1 1.5 2.7]
Tensor[shape={3}, stride={1}, Float64, CPU:0, 0x7f9ee3ec1070]
[0 1 2]
Tensor[shape={3}, stride={1}, Int32, CPU:0, 0x7f9ee3e9a160]


## Numpy I/O with direct memory map

In [20]:
# From numpy
np_a = np.ones((5,), dtype=np.int32)
o3_a = o3d.core.Tensor.from_numpy(np_a)
print(f"np_a: {np_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to numpy array reflects on open3d Tensor and vice versa
np_a[0] = 100
o3_a[1] = 200
print(f"np_a: {np_a}")
print(f"o3_a: {o3_a}")

np_a: [1 1 1 1 1]
o3_a: [1 1 1 1 1]
Tensor[shape={5}, stride={1}, Int32, CPU:0, 0x7f9ee3c9e5d0]



TypeError: _setitem(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.pybind.core.Tensor, arg0: open3d.pybind.core.TensorKey, arg1: open3d.pybind.core.Tensor) -> open3d.pybind.core.Tensor

Invoked with: [100 1 1 1 1]
Tensor[shape={5}, stride={1}, Int32, CPU:0, 0x7f9ee3c9e5d0], <open3d.pybind.core.TensorKey object at 0x10bdd3ea0>, 200

In [21]:
# To numpy
o3_a = o3d.core.Tensor([1, 1, 1, 1, 1], dtype=o3d.core.Dtype.Int32)
np_a = o3_a.numpy()

# Changes to numpy array reflects on open3d Tensor and vice versa
np_a[0] = 100
o3_a[1] = 200
print(f"np_a: {np_a}")
print(f"o3_a: {o3_a}")

# For CUDA Tensor, call cpu() before calling numpy()
o3_a = o3d.core.Tensor([1, 1, 1, 1, 1], device=o3d.core.Device("CUDA:0"))
print(f"\no3_a.cpu().numpy(): {o3_a.cpu().numpy()}")

TypeError: _setitem(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.pybind.core.Tensor, arg0: open3d.pybind.core.TensorKey, arg1: open3d.pybind.core.Tensor) -> open3d.pybind.core.Tensor

Invoked with: [100 1 1 1 1]
Tensor[shape={5}, stride={1}, Int32, CPU:0, 0x7f9ee3cb6dd0], <open3d.pybind.core.TensorKey object at 0x121e2c148>, 200

## PyTorch I/O with DLPack memory map

In [ ]:
# From PyTorch
th_a = torch.ones((5,)).cuda(0)
o3_a = o3d.core.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(th_a))
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to PyTorch array reflects on open3d Tensor and vice versa
th_a[0] = 100
o3_a[1] = 200
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")

In [ ]:
# To PyTorch
o3_a = o3d.core.Tensor([1, 1, 1, 1, 1], device=o3d.core.Device("CUDA:0"))
th_a = torch.utils.dlpack.from_dlpack(o3_a.to_dlpack())
o3_a = o3d.core.Tensor.from_dlpack(torch.utils.dlpack.to_dlpack(th_a))
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")
print("")

# Changes to PyTorch array reflects on open3d Tensor and vice versa
th_a[0] = 100
o3_a[1] = 200
print(f"th_a: {th_a}")
print(f"o3_a: {o3_a}")

## Binary element-wise operation: add, sub, mul, div, ...

In [22]:
a = o3d.core.Tensor([1, 1, 1], dtype=o3d.core.Dtype.Float32)
b = o3d.core.Tensor([2, 2, 2], dtype=o3d.core.Dtype.Float32)
print(a + b)
print(a - b)
print(a * b)
print(a / b)
print("")

# Automated broadcasting
a = o3d.core.Tensor(np.ones((2, 3)), dtype=o3d.core.Dtype.Float32)
b = o3d.core.Tensor(np.ones((3,)), dtype=o3d.core.Dtype.Float32)
print(a + b)

[3.0 3.0 3.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7f9ee3e9a160]
[-1.0 -1.0 -1.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7f9ee3c9c9d0]
[2.0 2.0 2.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7f9ee3c2fbb0]
[0.5 0.5 0.5]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7f9ee3cb1cd0]

[[2.0 2.0 2.0],
 [2.0 2.0 2.0]]
Tensor[shape={2, 3}, stride={3, 1}, Float32, CPU:0, 0x7f9ee3cb6db0]


## Unary element-wise operation: sqrt, sin, cos, ...

In [23]:
a = o3d.core.Tensor([4, 9, 16], dtype=o3d.core.Dtype.Float32)
print(a.sqrt())
print(a.sin())
print(a.cos())

[2.0 3.0 4.0]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7f9ee3cbe400]
[-0.756802 0.412118 -0.287903]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7f9ee8e3d3e0]
[-0.653644 -0.91113 -0.957659]
Tensor[shape={3}, stride={1}, Float32, CPU:0, 0x7f9ee8e4e690]


## Reduction: sum, prod, min, max

In [24]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3d.core.Tensor(vals)
print(a.sum())
print(a.prod())
print(a.min())
print(a.max())

276
Tensor[shape={}, stride={}, Int64, CPU:0, 0x7f9ee61c3c90]
0
Tensor[shape={}, stride={}, Int64, CPU:0, 0x7f9ee3eba9f0]
0
Tensor[shape={}, stride={}, Int64, CPU:0, 0x7f9ee3e65c20]
23
Tensor[shape={}, stride={}, Int64, CPU:0, 0x7f9ee3e1c260]


In [25]:
# With specified dimension
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3d.core.Tensor(vals)

print(a.sum(dim=(0)), "\n")
print(a.sum(dim=(0, 2)), "\n")
print(a.sum(dim=(0, 2), keepdim=True))

[[12 14 16 18],
 [20 22 24 26],
 [28 30 32 34]]
Tensor[shape={3, 4}, stride={4, 1}, Int64, CPU:0, 0x7f9ee61a3f80] 

[60 92 124]
Tensor[shape={3}, stride={1}, Int64, CPU:0, 0x7f9ee3cb6db0] 

[[[60],
  [92],
  [124]]]
Tensor[shape={1, 3, 1}, stride={3, 1, 1}, Int64, CPU:0, 0x7f9ee3cb6150]


## Slicing, indexing, getitem (returns a view), and setitem

In [26]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3d.core.Tensor(vals)

# Slicing __getitem__
print(a[1:], "\n")

# Indexing __getitem__
print(a[1, 2], "\n")

# Combined __getitem__
print(a[:-1, 0:3:2, 2])

[[[12 13 14 15],
  [16 17 18 19],
  [20 21 22 23]]]
Tensor[shape={1, 3, 4}, stride={12, 4, 1}, Int64, CPU:0, 0x7f9ee8e69420] 

[20 21 22 23]
Tensor[shape={4}, stride={1}, Int64, CPU:0, 0x7f9ee8e69460] 

[[2 10]]
Tensor[shape={1, 2}, stride={12, 8}, Int64, CPU:0, 0x7f9ee8e693d0]


In [27]:
# Example __setitem__
print(a, "\n")
a[:, :, 2] = 100
print(a)

[[[0 1 2 3],
  [4 5 6 7],
  [8 9 10 11]],
 [[12 13 14 15],
  [16 17 18 19],
  [20 21 22 23]]]
Tensor[shape={2, 3, 4}, stride={12, 4, 1}, Int64, CPU:0, 0x7f9ee8e693c0] 



TypeError: _setitem_vector(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.pybind.core.Tensor, arg0: List[open3d.pybind.core.TensorKey], arg1: open3d.pybind.core.Tensor) -> open3d.pybind.core.Tensor

Invoked with: [[[0 1 2 3],
  [4 5 6 7],
  [8 9 10 11]],
 [[12 13 14 15],
  [16 17 18 19],
  [20 21 22 23]]]
Tensor[shape={2, 3, 4}, stride={12, 4, 1}, Int64, CPU:0, 0x7f9ee8e693c0], [<open3d.pybind.core.TensorKey object at 0x121dfa618>, <open3d.pybind.core.TensorKey object at 0x121dfa768>, <open3d.pybind.core.TensorKey object at 0x121dfad88>], 100

## Advanced indexing

In [28]:
vals = np.array(range(24)).reshape((2, 3, 4))
a = o3d.core.Tensor(vals)

# __getitem__
print(a[:, [1, 2], [1, 2]])
print(a[1, [[1, 2], [2, 1]], 0:4:2])

# __setitem__
a[:, 0:2, [1, 2]] = o3d.core.Tensor(np.array([[100, 200], [300, 400]]))
print(a)

[[5 10],
 [17 22]]
Tensor[shape={2, 2}, stride={2, 1}, Int64, CPU:0, 0x7f9ee3cc03c0]
[[[16 18],
  [20 22]],
 [[20 22],
  [16 18]]]
Tensor[shape={2, 2, 2}, stride={4, 2, 1}, Int64, CPU:0, 0x7f9ee3eeb410]
[[[0 100 200 3],
  [4 300 400 7],
  [8 9 10 11]],
 [[12 100 200 15],
  [16 300 400 19],
  [20 21 22 23]]]
Tensor[shape={2, 3, 4}, stride={12, 4, 1}, Int64, CPU:0, 0x7f9ee3cb7140]


## TensorList

In [29]:
vals = np.array(range(24), dtype = np.float32).reshape((2, 3, 4))

# Empty TensorList
a = o3d.core.TensorList([3, 4])
print(a)

# TensorList with single Tensor
b = o3d.core.TensorList([3, 4], size = 1)
print(b)

# TensorList from tensor
c = o3d.core.TensorList.from_tensor(o3d.core.Tensor(vals))
print(c)

# TensorList from multiple tensors
d = o3d.core.TensorList.from_tensors([o3d.core.Tensor(vals[0]), o3d.core.Tensor(vals[1])])
print(d)

# Concatenate TensorLists
print(b + c)
print(o3d.core.TensorList.concat(b, c))

# Append a Tensor to TensorList
d.push_back(o3d.core.Tensor(vals[0]))
print(d)

# Append a TensorList to another TensorList
d.extend(b)
print(d)



TensorList[size=0, shape={3, 4}, Float32, CPU:0]

TensorList[size=1, shape={3, 4}, Float32, CPU:0]

TensorList[size=2, shape={3, 4}, Float32, CPU:0]

TensorList[size=2, shape={3, 4}, Float32, CPU:0]

TensorList[size=3, shape={3, 4}, Float32, CPU:0]

TensorList[size=3, shape={3, 4}, Float32, CPU:0]

TensorList[size=3, shape={3, 4}, Float32, CPU:0]

TensorList[size=4, shape={3, 4}, Float32, CPU:0]


## Logical operations

In [30]:
a = o3d.core.Tensor(np.array([True, False, True, False]))
b = o3d.core.Tensor(np.array([True, True, False, False]))

print(a.logical_and(b))
print(a.logical_or(b))
print(a.logical_xor(b))

[True False False False]
Tensor[shape={4}, stride={1}, Bool, CPU:0, 0x7f9ee3e96c80]
[True True True False]
Tensor[shape={4}, stride={1}, Bool, CPU:0, 0x7f9ee3fd20d0]
[False True True False]
Tensor[shape={4}, stride={1}, Bool, CPU:0, 0x7f9ee3fd20d0]


## Comparision Operations

In [31]:
a = o3d.core.Tensor([0, 1, -1])
b = o3d.core.Tensor([0, 0, 0])

print(a > b)
print(a >= b)
print(a < b)
print(a <= b)
print(a == b)
print(a != b)

[False True False]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x7f9ee3e18850]
[True True False]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x7f9ee60c6280]
[False False True]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x7f9ee60c6280]
[True False True]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x7f9ee60c6280]
[True False False]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x7f9ee60c6280]
[False True True]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x7f9ee60c6280]


## Nonzero operations

In [32]:
a = o3d.core.Tensor([[3, 0, 0], [0, 4, 0], [5, 6, 0]])

print(a.nonzero())
print(a.nonzero(as_tuple = 1))

[[0 1 2 2],
 [0 1 0 1]]
Tensor[shape={2, 4}, stride={4, 1}, Int64, CPU:0, 0x7f9ee6359d30]
[[0 1 2 2]
Tensor[shape={4}, stride={1}, Int64, CPU:0, 0x7f9ee3cb61b0], [0 1 0 1]
Tensor[shape={4}, stride={1}, Int64, CPU:0, 0x7f9ee3cbd490]]


## Boolean operations

In [33]:
a = o3d.core.Tensor([1, -1, -2, -3])

a[a < 0]
print(a[a < 0])


TypeError: lt(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.pybind.core.Tensor, arg0: open3d.pybind.core.Tensor) -> open3d.pybind.core.Tensor

Invoked with: [1 -1 -2 3]
Tensor[shape={4}, stride={1}, Int64, CPU:0, 0x7f9ee3cb61b0], 0

## Scalar operations

In [34]:
a = o3d.core.Tensor.ones((2, 3), dtype = o3d.core.Dtype.Float32)

print(a.add(1))
print(a + 1)
print(a + True)
print(a - 1)
print(a - True)
print(a * 10)
print(10 / a)

# Inplace
a.add_(1)
print(a)
a.sub_(1)
print(a)

# Shorthand
a += 1
print(a)
a -= 1
print(a)
a *= 10
print(a)
a /= 2
print(a)

[[2.0 2.0 2.0],
 [2.0 2.0 2.0]]
Tensor[shape={2, 3}, stride={3, 1}, Float32, CPU:0, 0x7f9ee3e7a5e0]
[[2.0 2.0 2.0],
 [2.0 2.0 2.0]]
Tensor[shape={2, 3}, stride={3, 1}, Float32, CPU:0, 0x7f9ee3fecb60]
[[2.0 2.0 2.0],
 [2.0 2.0 2.0]]
Tensor[shape={2, 3}, stride={3, 1}, Float32, CPU:0, 0x7f9ee3fecb60]
[[0.0 0.0 0.0],
 [0.0 0.0 0.0]]
Tensor[shape={2, 3}, stride={3, 1}, Float32, CPU:0, 0x7f9ee3fecb60]
[[0.0 0.0 0.0],
 [0.0 0.0 0.0]]
Tensor[shape={2, 3}, stride={3, 1}, Float32, CPU:0, 0x7f9ee3fecb60]
[[10.0 10.0 10.0],
 [10.0 10.0 10.0]]
Tensor[shape={2, 3}, stride={3, 1}, Float32, CPU:0, 0x7f9ee3fecb60]
[[10.0 10.0 10.0],
 [10.0 10.0 10.0]]
Tensor[shape={2, 3}, stride={3, 1}, Float32, CPU:0, 0x7f9ee3fecb60]
[[2.0 2.0 2.0],
 [2.0 2.0 2.0]]
Tensor[shape={2, 3}, stride={3, 1}, Float32, CPU:0, 0x7f9ee3e1a3e0]
[[1.0 1.0 1.0],
 [1.0 1.0 1.0]]
Tensor[shape={2, 3}, stride={3, 1}, Float32, CPU:0, 0x7f9ee3e1a3e0]
[[2.0 2.0 2.0],
 [2.0 2.0 2.0]]
Tensor[shape={2, 3}, stride={3, 1}, Float32, CPU:0, 0x7f